# Multiverse Knowledge

Make sure the [Multiverse Framework](https://github.com/Multiverse-Framework/Multiverse) is installed properly.

## Introduction

In cognition-enabled robotics, tasks like "prepare a breakfast" require robots to infer specific actions from environmental data, relying heavily on knowledge and reasoning. However, this process is limited by incompatible data formats, redundant data, and a lack of semantic information. Therefore, we take advantage of the
Universal Scene Description (USD) format which is an emerging standard for the authoring, visualization and simulation of complex environments. In this tutorial, we investigate the conversion of USD-based environment models into Knowledge Graph (KG) representations that facilitate semantic querying and integration with
additional knowledge sources.

The first step in our method for generating a KG from a USD scene is to establish a USD layer containing class prims representing the TBox ontology. Another layer representing the scene graph imports the TBox USD layer and uses a custom API to tag prims with ontological concepts. The semantic USD scene graph are then translated into the KG.

![multiverse_knowledge_I1.png](images/multiverse_knowledge_I1.png)

## I. Generate a USD layer from an ontology for semantic tagging

The first step in the process for generating a KG from a USD scene is to establish a USD layer containing class prims representing the TBox ontology.

In [1]:
RESOURCES=$PWD/../src/ontology
OUTPUT=$PWD/../output

WORLD="dlr_kitchen" 

INPUT_OWL_PATH=$RESOURCES/SOMA_DFL_module.owl
OUTPUT_USD_PATH=$OUTPUT/$OBJECT/usda/${WORLD}_TBox.usda

# Depending on the size of the ontology, this could take a while
TBox_to_usd --in_owl=$INPUT_OWL_PATH --out_usd=$OUTPUT_USD_PATH

The result should appear as follows:

![multiverse_knowledge_I.png](images/multiverse_knowledge_I.png)

## II. Run automatic semantic reporting and semantic tagging based on the object names

Before appling semantic information, we enhance labeling with semantic reports. We use the FRED tool to link object names in the scene to entities in the knowledge graph, and when links are missing, we apply heuristic methods.

The semantic tagging is deployed based on the semantic reporting afterwards.

In [2]:
RESOURCES=$PWD/../resources
OUTPUT=$PWD/../output

WORLD="dlr_kitchen" 

INPUT_USD_PATH=$RESOURCES/$WORLD/$WORLD.usda
INPUT_TBOX_USD_PATH=$OUTPUT/$WORLD/usda/${WORLD}_TBox.usda
OUTPUT_USD_PATH=$OUTPUT/$WORLD/usda/${WORLD}.usda

# Run semantic reporting with FRED
semantic_reporting --in_usd=$INPUT_USD_PATH --in_TBox_usd=$INPUT_TBOX_USD_PATH --out_usd=$OUTPUT_USD_PATH

# Run automatic semantic tagging
python $PWD/../src/semantic_tagging.py --in_usd=$OUTPUT_USD_PATH --in_TBox_usd=$INPUT_TBOX_USD_PATH --out_ABox_usd=$OUTPUT_USD_PATH

Loading semantic reporting module...
Found ['http://www.ease-crc.org/ont/SOMA_DFL.owl#kitchen.n.wn.artifact'] for dlr_kitchen
Found ['http://www.ease-crc.org/ont/SOMA_DFL.owl#kitchenette.n.wn.artifact'] for kitchenette
Semantic path /SOMA_DFL/_class_kitchenettenwnartifact not found in TBox
Found ['http://www.ease-crc.org/ont/SOMA_DFL.owl#drawer.n.wn.artifact'] for drawer_02
Found ['http://www.ease-crc.org/ont/SOMA_DFL.owl#drawer.n.wn.artifact', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#handle.n.wn.artifact'] for drawer_02_handle
Found ['http://www.ease-crc.org/ont/SOMA_DFL.owl#drawer.n.wn.artifact'] for drawer_03
Found ['http://www.ease-crc.org/ont/SOMA_DFL.owl#drawer.n.wn.artifact', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#handle.n.wn.artifact'] for drawer_03_handle
Found ['http://www.ease-crc.org/ont/SOMA_DFL.owl#door.n.wn.artifact..movable_barrier', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#doorway.n.wn.artifact', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#oven.n.wn.artifact'] for o

The result should appear as follows:

![multiverse_knowledge_II.png](images/multiverse_knowledge_II.png)

## III. Use multiverse_view to for semantic labeling

After completing semantic reporting and tagging, we should verify the accuracy of our semantic scene graph and make any necessary adjustments to the semantic data. This involves cleaning up the USD scene graph for size optimization and using our visualization tool, [multiverse_view](https://multiverse-framework.github.io/Multiverse-View/), to review the labels.

### 1. Flatten the above usd file into a single layer

In [3]:
OUTPUT=$PWD/../output

WORLD="dlr_kitchen" 

INPUT_USD_PATH=$OUTPUT/$WORLD/usda/${WORLD}.usda
OUTPUT_USD_PATH=$OUTPUT/$WORLD/usda/${WORLD}_flatten.usda

usdcat $INPUT_USD_PATH -o $OUTPUT_USD_PATH --flatten
python $PWD/../src/clean_up_usd.py --in_usd=$OUTPUT_USD_PATH --out_usd=$OUTPUT_USD_PATH

Writing to /media/giangnguyen/Storage/Multiverse-Docker/images/Multiverse-Tutorials-image/Multiverse-Tutorials/tutorials/../output/dlr_kitchen/usda/dlr_kitchen_flatten.usda


The result should appear as follows:

![multiverse_knowledge_III1.png](images/multiverse_knowledge_III1.png)

### 2. Upload the above usd file to https://multiverse-framework.github.io/Multiverse-View/ and tag the semantic labels

The result should appear as follows:

![multiverse_knowledge_III2.png](images/multiverse_knowledge_III2.png)

### 3. Download the semantic USD file

Download the semantic USD file by clicking on `Download` button, then move it into the output folder for further steps. You could skip this step if the semantic USD file is fine.

## IV. Convert the semantic USD scene into a knowledge graph

Finally, the semantic USD scene can be converted into a knowledge graph for robots.

In [4]:
RESOURCES=$PWD/../src/ontology
OUTPUT=$PWD/../output

WORLD="dlr_kitchen"

INPUT_USD_PATH=$OUTPUT/$WORLD/usda/${WORLD}_semantic_flatten.usda
INPUT_OWL_PATH=$RESOURCES/SOMA_DFL_module.owl
OUTPUT_OWL_PATH=$OUTPUT/$WORLD/owl/${WORLD}.owl

usd_to_ABox --in_usd=$INPUT_USD_PATH --in_owl=$INPUT_OWL_PATH --out_owl=$OUTPUT_OWL_PATH

Save ABox ontology to /media/giangnguyen/Storage/Multiverse-Docker/images/Multiverse-Tutorials-image/Multiverse-Tutorials/tutorials/../output/dlr_kitchen/owl/dlr_kitchen.owl, from 143 prims to 22 xforms, 103 gprims, 9 joints and excludes 9 others
/media/giangnguyen/Storage/Multiverse-Docker/images/Multiverse-Tutorials-image/Multiverse-Tutorials/tutorials/../output/dlr_kitchen/owl/dlr_kitchen.owl:
  number of nodes: 1095
  number of edges: 3744
  number of classes: 0
  number of instances: 799


The result should appear as follows:

![multiverse_knowledge_IV.png](images/multiverse_knowledge_IV.png)